In [1]:
# set random seed
import random
random.seed(42)
import numpy as np
np.random.seed(42)
import torch
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Parameters

In [2]:
# import pyepo
# generate data
grid = (5,5) # grid size
num_data = 100 # number of training data
num_feat = 5 # size of feature
num_test = 1000
deg = 1.2 # polynomial degree
e = 0.5 # noise width

# Generate Data

In [3]:
from Data import data_generation
data_gen = data_generation()

#  ****** Data generation process is the same as SPO+ *********
feats, costs = data_gen.generate_Shortest_Path_Data(num_data+num_test, num_feat, grid, deg, e, seed=42)

#  ****** Data generation process is the same as DDR *********
# lower = 0
# upper = 1
# p = 5
# d = 40
# alpha = 1
# mis = 4
# n_epsilon = 1
# W_star = data_gen.generate_truth("",lower, upper, p, d, version = 0) 
# x_test, z_test_ori, c_test, x_train, z_train_ori, c_train, W_star = data_gen.generate_samples("",p, d, num_test, num_data, alpha, W_star, n_epsilon, mis, thres = 10, 
#                         version = 1, x_dist = 'normal', e_dist = 'normal', x_low = 0, x_up = 2, x_mean = 2, x_var = 0.25, bump = 0) 

# split train test data
from sklearn.model_selection import train_test_split
x_train, x_test, c_train, c_test = train_test_split(feats, costs, test_size=num_test, random_state=42)

### SPO+ 

In [4]:
from SPO_Plus import run_SPO_Shortest_Path
SPO_runner = run_SPO_Shortest_Path()
batch_size = 20
num_epochs = 1
arcs,loader_train,loader_test,cost_Oracle,cost_SPO = SPO_runner.run(x_train,c_train,x_test,c_test,batch_size,num_feat,grid,num_epochs,True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-25
Test
Optimizing for optDataset...


100%|██████████| 100/100 [00:00<00:00, 1854.46it/s]


Test
Optimizing for optDataset...


100%|██████████| 1000/1000 [00:00<00:00, 2075.47it/s]


oracle cost =  6.6683534908294675
Num of cores: 2
epoch 0: Average SPO Cost =  8.279449820011854
epoch =  0  Average SPO Cost =  8.206841227144002


# OLS

In [5]:
from OLS import run_OLS_Shortest_Path
OLS_runner = run_OLS_Shortest_Path()
cost_OLS = OLS_runner.run(arcs,x_train,c_train,grid,loader_test)
print("Average OLS Cost = ",np.mean(cost_OLS))

Average OLS Cost =  7.6799817386567595


# DDR

In [9]:
print("Average Oracle Cost = ",np.mean(cost_Oracle),"Std = ", np.std(cost_Oracle))
print("Average SPO Cost = ", np.mean(cost_SPO),"Std = ", np.std(cost_SPO))
print("Average OLS Cost = ", np.mean(cost_OLS),"Std = ", np.std(cost_OLS))

Average Oracle Cost =  6.6683534908294675 Std =  1.2185374400285396
Average SPO Cost =  8.206841227144002 Std =  1.4921237823474969
Average OLS Cost =  7.6799817386567595 Std =  1.5318926993671838


In [ ]:
from DDR import run_DDR_Shortest_Path
DDR_runner = run_DDR_Shortest_Path()

mu_arr = np.arange(0.1,1,0.05)
lamb_arr = np.arange(0.1,1,0.05)
lamb_arr = [0.1]
minimum_value = 1000000000
for lamb in lamb_arr:
    print("======== lambda = ",lamb,"============")
    for mu in mu_arr:
        num_nodes = 25
        # w0_ddr_val,W_ddr_val = solve_DDR(lamb,mu,num_nodes,x_train,c_train)
        cost_DDR = DDR_runner.run(arcs,x_train, c_train, grid,loader_test,lamb,mu,num_nodes)
        if np.mean(cost_DDR) < minimum_value:
            minimum_value = np.mean(cost_DDR)
            print("lambda = ",lamb, ", mu = ",mu, ", Lowerst Average DDR Cost = ",minimum_value, " Std = ",np.std(cost_DDR))
        # print("lambda = ",lamb, ", mu = ",mu, ",Average DDR Cost = ",np.mean(cost_DDR), " Std = ",np.std(cost_DDR))

======== lambda =  0.1 ============
lambda =  0.1 , mu =  0.1 , Lowerst Average DDR Cost =  7.69086926355958  Std =  1.5374735989970505
lambda =  0.1 , mu =  0.15000000000000002 , Lowerst Average DDR Cost =  7.6902248392403125  Std =  1.5374344489144305
lambda =  0.1 , mu =  0.20000000000000004 , Lowerst Average DDR Cost =  7.683062559485435  Std =  1.5288134090938752
lambda =  0.1 , mu =  0.3500000000000001 , Lowerst Average DDR Cost =  7.680836561828852  Std =  1.5359545452277896
lambda =  0.1 , mu =  0.6000000000000002 , Lowerst Average DDR Cost =  7.68034748211503  Std =  1.5321031530001343
lambda =  0.1 , mu =  0.9000000000000002 , Lowerst Average DDR Cost =  7.677918752789497  Std =  1.5298038000216558
lambda =  0.1 , mu =  0.9500000000000003 , Lowerst Average DDR Cost =  7.676757207006216  Std =  1.5304983332245425
